In [1]:
import os
import re
import pandas as pd

from math import floor

In [65]:
directory = 'pipeline_intermediary/denovo_output/'
files = [f for f in os.listdir(directory) if bool(re.search(r'_out.txt', f))]
selection = 0.25
samples = []

def summary_times(group):
    scans = group['scan']
    return len(scans)

def summary_scans(group):
    scans = group['scan']
    return "_".join(scans)

def summary_rank(group):
    ranks = group['rank']
    return min(ranks)

In [66]:
for file in files:
    xx = pd.read_csv(directory + file, sep = "\t", header = 0)

    xx['output_seq'] = [re.sub(",", "", str(peptide)) for peptide in xx['output_seq']]
    xx['output_seq'] = [re.sub("mod", "", str(peptide)) for peptide in xx['output_seq']]
    xx = xx.loc[xx['output_score'] != float("Inf")]

    xx = xx.sort_values('output_score', ascending = False)
    xx = xx.head(floor(selection * floor(len(xx.index))))
    #xx = xx[['scan', 'output_seq']]

    xx['pep_length'] = [len(peptide) for peptide in xx['output_seq']]
    xx = xx.loc[(xx['pep_length'] >= 10) & (xx['pep_length'] <= 17)]
    xx['rank'] = list(range(1, len(xx.index) + 1))

    grouped = xx.groupby('output_seq')

    summary = grouped.apply(summary_times).to_frame()
    summary['output_seq'] = summary.index
    summary.columns = ['times', 'output_seq']
    summary = summary[['output_seq', 'times']]
    summary['rank'] = grouped.apply(summary_rank)
    summary['scans'] = grouped.apply(summary_scans)

    samples += [summary]


In [72]:
for summary in samples:
    nms = [">S" + summary['scans'][i] + "_" + str(summary['times'][i]) for i in range(len(summary))]
    to_write = [None]*2*len(nms)
    to_write[::2] = nms
    to_write[1::2] = summary['output_seq']

    with open('testing_combined.fasta', 'a') as fasta_file:
        for line in to_write:
            fasta_file.write(f"{line}\n")


In [3]:
os.listdir('.')

['config.ini',
 'diamond-2.0.15',
 'example.ini',
 'Kaiko_2.py',
 'Kaiko_3.py',
 'Kaiko_4.py',
 'kaiko_combined_denovo_outputs.ipynb',
 'kaiko_defaults.ini',
 'Kaiko_denovo',
 'Kaiko_pipeline_main.py',
 'MinT_samplesgfda_01.dmd',
 'ncbi_taxa_2022-10-20',
 'Parsing.py',
 'pipeline_input',
 'pipeline_intermediary',
 'pipeline_output',
 'testing.csv',
 'testing_combined.fasta',
 'uniref100.fasta.gz',
 '__pycache__']